## Assignment 4 - Helper Functions

We begin by including the functions to generate frequent itemsets (via the Apriori algorithm) and resulting association rules:

In [2]:
# (c) 2016 Everaldo Aguiar & Reid Johnson
#
# Modified from:
# Marcel Caraciolo (https://gist.github.com/marcelcaraciolo/1423287)
#
# Functions to compute and extract association rules from a given frequent itemset 
# generated by the Apriori algorithm.
#
# The Apriori algorithm is defined by Agrawal and Srikant in:
# Fast algorithms for mining association rules
# Proc. 20th int. conf. very large data bases, VLDB. Vol. 1215. 1994
import csv
import numpy as np

def load_dataset(filename):
    '''Loads an example of market basket transactions from a provided csv file.

    Returns: A list (database) of lists (transactions). Each element of a transaction is 
    an item.
    '''

    with open(filename,'r') as dest_f:
        data_iter = csv.reader(dest_f, delimiter = ',', quotechar = '"')
        data = [data for data in data_iter]
        data_array = np.asarray(data)
        
    return data_array

def apriori(dataset, min_support=0.5, verbose=False):
    """Implements the Apriori algorithm.

    The Apriori algorithm will iteratively generate new candidate 
    k-itemsets using the frequent (k-1)-itemsets found in the previous 
    iteration.

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate 
        candidate itemsets.

    min_support : float
        The minimum support threshold. Defaults to 0.5.

    Returns
    -------
    F : list
        The list of frequent itemsets.

    support_data : dict
        The support data for all candidate itemsets.

    References
    ----------
    .. [1] R. Agrawal, R. Srikant, "Fast Algorithms for Mining Association 
           Rules", 1994.

    """
    C1 = create_candidates(dataset)
    D = list(map(set, dataset))
    F1, support_data = support_prune(D, C1, min_support, verbose=False) # prune candidate 1-itemsets
    F = [F1] # list of frequent itemsets; initialized to frequent 1-itemsets
    k = 2 # the itemset cardinality
    while (len(F[k - 2]) > 0):
        Ck = apriori_gen(F[k-2], k) # generate candidate itemsets
        Fk, supK = support_prune(D, Ck, min_support) # prune candidate itemsets
        support_data.update(supK) # update the support counts to reflect pruning
        F.append(Fk) # add the pruned candidate itemsets to the list of frequent itemsets
        k += 1

    if verbose:
        # Print a list of all the frequent itemsets.
        for kset in F:
            for item in kset:
                print("" \
                    + "{" \
                    + "".join(str(i) + ", " for i in iter(item)).rstrip(', ') \
                    + "}" \
                    + ":  sup = " + str(round(support_data[item], 3)))

    return F, support_data

def create_candidates(dataset, verbose=False):
    """Creates a list of candidate 1-itemsets from a list of transactions.

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate candidate 
        itemsets.

    Returns
    -------
    The list of candidate itemsets (c1) passed as a frozenset (a set that is 
    immutable and hashable).
    """
    c1 = [] # list of all items in the database of transactions
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()

    if verbose:
        # Print a list of all the candidate items.
        print("" \
            + "{" \
            + "".join(str(i[0]) + ", " for i in iter(c1)).rstrip(', ') \
            + "}")

    # Map c1 to a frozenset because it will be the key of a dictionary.
    return list(map(frozenset, c1))

def support_prune(dataset, candidates, min_support, verbose=False):
    """Returns all candidate itemsets that meet a minimum support threshold.

    By the apriori principle, if an itemset is frequent, then all of its 
    subsets must also be frequent. As a result, we can perform support-based 
    pruning to systematically control the exponential growth of candidate 
    itemsets. Thus, itemsets that do not meet the minimum support level are 
    pruned from the input list of itemsets (dataset).

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate candidate 
        itemsets.

    candidates : frozenset
        The list of candidate itemsets.

    min_support : float
        The minimum support threshold.

    Returns
    -------
    retlist : list
        The list of frequent itemsets.

    support_data : dict
        The support data for all candidate itemsets.
    """
    sscnt = {} # set for support counts
    for tid in dataset:
        for can in candidates:
            if can.issubset(tid):
                sscnt.setdefault(can, 0)
                sscnt[can] += 1

    num_items = float(len(dataset)) # total number of transactions in the dataset
    retlist = [] # array for unpruned itemsets
    support_data = {} # set for support data for corresponding itemsets
    for key in sscnt:
        # Calculate the support of itemset key.
        support = sscnt[key] / num_items
        if support >= min_support:
            retlist.insert(0, key)
        support_data[key] = support

    # Print a list of the pruned itemsets.
    if verbose:
        for kset in retlist:
            for item in kset:
                print("{" + str(item) + "}")
        print("")
        for key in sscnt:
            print("" \
                + "{" \
                + "".join([str(i) + ", " for i in iter(key)]).rstrip(', ') \
                + "}" \
                + ":  sup = " + str(support_data[key]))

    return retlist, support_data

def apriori_gen(freq_sets, k):
    """Generates candidate itemsets (via the F_k-1 x F_k-1 method).

    This operation generates new candidate k-itemsets based on the frequent 
    (k-1)-itemsets found in the previous iteration. The candidate generation 
    procedure merges a pair of frequent (k-1)-itemsets only if their first k-2 
    items are identical.

    Parameters
    ----------
    freq_sets : list
        The list of frequent (k-1)-itemsets.

    k : integer
        The cardinality of the current itemsets being evaluated.

    Returns
    -------
    retlist : list
        The list of merged frequent itemsets.
    """
    retList = [] # list of merged frequent itemsets
    lenLk = len(freq_sets) # number of frequent itemsets
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            a=list(freq_sets[i])
            b=list(freq_sets[j])
            a.sort()
            b.sort()
            F1 = a[:k-2] # first k-2 items of freq_sets[i]
            F2 = b[:k-2] # first k-2 items of freq_sets[j]

            if F1 == F2: # if the first k-2 items are identical
                # Merge the frequent itemsets.
                retList.append(freq_sets[i] | freq_sets[j])

    return retList

def rules_from_conseq(freq_set, H, support_data, rules, min_confidence=0.5, verbose=False):
    """Generates a set of candidate rules.

    Parameters
    ----------
    freq_set : frozenset
        The complete list of frequent itemsets.

    H : list
        A list of frequent itemsets (of a particular length).

    support_data : dict
        The support data for all candidate itemsets.

    rules : list
        A potentially incomplete set of candidate rules above the minimum 
        confidence threshold.

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.5.
    """
    m = len(H[0])
    if m == 1:
        Hmp1 = calc_confidence(freq_set, H, support_data, rules, min_confidence, verbose)
    if (len(freq_set) > (m+1)):
        Hmp1 = apriori_gen(H, m+1) # generate candidate itemsets
        Hmp1 = calc_confidence(freq_set, Hmp1, support_data, rules, min_confidence, verbose)
        if len(Hmp1) > 1:
            # If there are candidate rules above the minimum confidence 
            # threshold, recurse on the list of these candidate rules.
            rules_from_conseq(freq_set, Hmp1, support_data, rules, min_confidence, verbose)

def calc_confidence(freq_set, H, support_data, rules, min_confidence=0.5, verbose=False):
    """Evaluates the generated rules.

    One measurement for quantifying the goodness of association rules is 
    confidence. The confidence for a rule 'P implies H' (P -> H) is defined as 
    the support for P and H divided by the support for P 
    (support (P|H) / support(P)), where the | symbol denotes the set union 
    (thus P|H means all the items in set P or in set H).

    To calculate the confidence, we iterate through the frequent itemsets and 
    associated support data. For each frequent itemset, we divide the support 
    of the itemset by the support of the antecedent (left-hand-side of the 
    rule).

    Parameters
    ----------
    freq_set : frozenset
        The complete list of frequent itemsets.

    H : list
        A list of frequent itemsets (of a particular length).

    min_support : float
        The minimum support threshold.

    rules : list
        A potentially incomplete set of candidate rules above the minimum 
        confidence threshold.

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.5.

    Returns
    -------
    pruned_H : list
        The list of candidate rules above the minimum confidence threshold.
    """
    pruned_H = [] # list of candidate rules above the minimum confidence threshold
    for conseq in H: # iterate over the frequent itemsets
        conf = support_data[freq_set] / support_data[freq_set - conseq]
        if conf >= min_confidence:
            rules.append((freq_set - conseq, conseq, conf))
            pruned_H.append(conseq)

            if verbose:
                print("" \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(freq_set-conseq)]).rstrip(', ') \
                    + "}" \
                    + " ---> " \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(conseq)]).rstrip(', ') \
                    + "}" \
                    + ":  conf = " + str(round(conf, 3)) \
                    + ", sup = " + str(round(support_data[freq_set], 3)))

    return pruned_H

def generate_rules(F, support_data, min_confidence=0.5, verbose=True):
    """Generates a set of candidate rules from a list of frequent itemsets.

    For each frequent itemset, we calculate the confidence of using a
    particular item as the rule consequent (right-hand-side of the rule). By 
    testing and merging the remaining rules, we recursively create a list of 
    pruned rules.

    Parameters
    ----------
    F : list
        A list of frequent itemsets.

    support_data : dict
        The corresponding support data for the frequent itemsets (L).

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.5.

    Returns
    -------
    rules : list
        The list of candidate rules above the minimum confidence threshold.
    """
    rules = []
    for i in range(1, len(F)):
        for freq_set in F[i]:
            H1 = [frozenset([itemset]) for itemset in freq_set]
            if (i > 1):
                rules_from_conseq(freq_set, H1, support_data, rules, min_confidence, verbose)
            else:
                calc_confidence(freq_set, H1, support_data, rules, min_confidence, verbose)

    return rules

### To load our dataset of grocery transactions, use the command below

In [3]:
dataset = load_dataset('grocery.csv')
D = list(map(set, dataset))

### _dataset_ is now a ndarray containing each of the 9835 transactions

In [4]:
type(dataset)

numpy.ndarray

In [5]:
dataset.shape

(9835,)

In [6]:
dataset[0]

['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups']

In [7]:
dataset[1]

['tropical fruit', 'yogurt', 'coffee']

### _D_ Contains that dataset in a set format (which excludes duplicated items and sorts them)

In [8]:
type(D[0])

set

In [9]:
D[0]

{'citrus fruit', 'margarine', 'ready soups', 'semi-finished bread'}

# Complete the assignment below by making use of the provided funtions. 

## You may use the notebook file attached with lesson 3 as a reference

In [41]:
# Generate candidate itemsets.
C1 = create_candidates(dataset, verbose=True) # candidate 1-itemsets
#  Total 169 items are the candidates of 'grocery.csv' dataset.

{Instant food products, UHT-milk, abrasive cleaner, artif. sweetener, baby cosmetics, baby food, bags, baking powder, bathroom cleaner, beef, berries, beverages, bottled beer, bottled water, brandy, brown bread, butter, butter milk, cake bar, candles, candy, canned beer, canned fish, canned fruit, canned vegetables, cat food, cereals, chewing gum, chicken, chocolate, chocolate marshmallow, citrus fruit, cleaner, cling film/bags, cocoa drinks, coffee, condensed milk, cooking chocolate, cookware, cream, cream cheese , curd, curd cheese, decalcifier, dental care, dessert, detergent, dish cleaner, dishes, dog food, domestic eggs, female sanitary products, finished products, fish, flour, flower (seeds), flower soil/fertilizer, frankfurter, frozen chicken, frozen dessert, frozen fish, frozen fruits, frozen meals, frozen potato products, frozen vegetables, fruit/vegetable juice, grapes, hair spray, ham, hamburger meat, hard cheese, herbs, honey, house keeping products, hygiene articles, ice c

In [11]:
# Prune candidate 1-itemsets via support-based pruning to generate frequent 1-itemsets.
F1, support_data = support_prune(D, C1, 0.2, verbose=True)
#  passing min_support as 20%

{whole milk}

{citrus fruit}:  sup = 0.08276563294356888
{margarine}:  sup = 0.05856634468734113
{ready soups}:  sup = 0.0018301982714794102
{semi-finished bread}:  sup = 0.017691916624300967
{coffee}:  sup = 0.05805795627859685
{tropical fruit}:  sup = 0.10493136756481952
{yogurt}:  sup = 0.13950177935943062
{whole milk}:  sup = 0.25551601423487547
{cream cheese}:  sup = 0.03965429588205389
{meat spreads}:  sup = 0.004270462633451958
{pip fruit}:  sup = 0.07564819522114896
{condensed milk}:  sup = 0.010269445856634469
{long life bakery product}:  sup = 0.037417386883579054
{other vegetables}:  sup = 0.1934926283680732
{abrasive cleaner}:  sup = 0.0035587188612099642
{butter}:  sup = 0.05541433655312659
{rice}:  sup = 0.007625826131164209
{rolls/buns}:  sup = 0.18393492628368074
{UHT-milk}:  sup = 0.03345195729537367
{bottled beer}:  sup = 0.08052872394509406
{liquor (appetizer)}:  sup = 0.007930859176410779
{pot plants}:  sup = 0.01728520589730554
{cereals}:  sup = 0.00569395017793594

In [12]:
# Generate all the frequent itemsets using the Apriori algorithm.
F, support_data = apriori(dataset, min_support=0.2, verbose=True)
#  passing min_support as 20%

{whole milk}:  sup = 0.256


In [13]:
# Generate the association rules from a list of frequent itemsets.
H = generate_rules(F, support_data, min_confidence=0.2, verbose=True)
#  passing min_confidence as 20%

In [43]:
# Generate all the frequent itemsets using the Apriori algorithm.
F, support_data = apriori(dataset, min_support=0.15, verbose=True)
#  passing min_support as 15%
# Found 4 frequent items

{soda}:  sup = 0.174
{rolls/buns}:  sup = 0.184
{other vegetables}:  sup = 0.193
{whole milk}:  sup = 0.256


In [44]:
# Generate the association rules from a list of frequent itemsets.
H = generate_rules(F, support_data, min_confidence=0, verbose=True)
#  passing min_confidence as 0%
# Even with confidence 0%, there is no association rule as freuent set consist of single items

In [45]:
# Generate all the frequent itemsets using the Apriori algorithm.
F, support_data = apriori(dataset, min_support=0.1, verbose=True)
#  passing min_support as 10%
# Found 8 frequent items

{root vegetables}:  sup = 0.109
{soda}:  sup = 0.174
{bottled water}:  sup = 0.111
{rolls/buns}:  sup = 0.184
{other vegetables}:  sup = 0.193
{whole milk}:  sup = 0.256
{yogurt}:  sup = 0.14
{tropical fruit}:  sup = 0.105


In [17]:
# Generate the association rules from a list of frequent itemsets.
H = generate_rules(F, support_data, min_confidence=0.1, verbose=True)
#  passing min_confidence as 10%

In [18]:
# Generate all the frequent itemsets using the Apriori algorithm.
F, support_data = apriori(dataset, min_support=0.09, verbose=True)
#  passing min_support as 9%
# Found 10 frequent items

{shopping bags}:  sup = 0.099
{sausage}:  sup = 0.094
{root vegetables}:  sup = 0.109
{soda}:  sup = 0.174
{bottled water}:  sup = 0.111
{rolls/buns}:  sup = 0.184
{other vegetables}:  sup = 0.193
{whole milk}:  sup = 0.256
{yogurt}:  sup = 0.14
{tropical fruit}:  sup = 0.105


In [19]:
# Generate the association rules from a list of frequent itemsets.
H = generate_rules(F, support_data, min_confidence=0, verbose=True)
#  passing min_confidence as 0%
# Even with confidence 0%, there is no association rule as freuent set consist of single items

In [20]:
# Generate all the frequent itemsets using the Apriori algorithm.
F, support_data = apriori(dataset, min_support=0.08, verbose=True)
#  passing min_support as 8%
# Found 13 frequent items

{shopping bags}:  sup = 0.099
{sausage}:  sup = 0.094
{root vegetables}:  sup = 0.109
{pastry}:  sup = 0.089
{soda}:  sup = 0.174
{bottled water}:  sup = 0.111
{bottled beer}:  sup = 0.081
{rolls/buns}:  sup = 0.184
{other vegetables}:  sup = 0.193
{whole milk}:  sup = 0.256
{yogurt}:  sup = 0.14
{tropical fruit}:  sup = 0.105
{citrus fruit}:  sup = 0.083


In [21]:
# Generate the association rules from a list of frequent itemsets.
H = generate_rules(F, support_data, min_confidence=0.08, verbose=True)
#  passing min_confidence as 8%

In [22]:
# Generate all the frequent itemsets using the Apriori algorithm.
F, support_data = apriori(dataset, min_support=0.06, verbose=True)
#  passing min_support as 6%
# Found 21 frequent items

{domestic eggs}:  sup = 0.063
{whipped/sour cream}:  sup = 0.072
{shopping bags}:  sup = 0.099
{brown bread}:  sup = 0.065
{sausage}:  sup = 0.094
{canned beer}:  sup = 0.078
{root vegetables}:  sup = 0.109
{pastry}:  sup = 0.089
{newspapers}:  sup = 0.08
{fruit/vegetable juice}:  sup = 0.072
{soda}:  sup = 0.174
{bottled water}:  sup = 0.111
{bottled beer}:  sup = 0.081
{rolls/buns}:  sup = 0.184
{other vegetables}:  sup = 0.193
{pip fruit}:  sup = 0.076
{whole milk}:  sup = 0.256
{yogurt}:  sup = 0.14
{tropical fruit}:  sup = 0.105
{citrus fruit}:  sup = 0.083
{whole milk, other vegetables}:  sup = 0.075


In [23]:
# Generate the association rules from a list of frequent itemsets.
H = generate_rules(F, support_data, min_confidence=0.06, verbose=True)
#  At 6% we have found first 2 association rules, which makes sense a person who eats vegetables also likes 
#  to have whole milk

{other vegetables} ---> {whole milk}:  conf = 0.387, sup = 0.075
{whole milk} ---> {other vegetables}:  conf = 0.293, sup = 0.075


In [24]:
# Generate all the frequent itemsets using the Apriori algorithm.
F, support_data = apriori(dataset, min_support=0.04, verbose=True)
#  passing min_support as 4%

{frozen vegetables}:  sup = 0.048
{domestic eggs}:  sup = 0.063
{whipped/sour cream}:  sup = 0.072
{pork}:  sup = 0.058
{napkins}:  sup = 0.052
{shopping bags}:  sup = 0.099
{brown bread}:  sup = 0.065
{sausage}:  sup = 0.094
{canned beer}:  sup = 0.078
{root vegetables}:  sup = 0.109
{pastry}:  sup = 0.089
{newspapers}:  sup = 0.08
{fruit/vegetable juice}:  sup = 0.072
{chicken}:  sup = 0.043
{soda}:  sup = 0.174
{frankfurter}:  sup = 0.059
{beef}:  sup = 0.052
{curd}:  sup = 0.053
{white bread}:  sup = 0.042
{chocolate}:  sup = 0.05
{bottled water}:  sup = 0.111
{bottled beer}:  sup = 0.081
{rolls/buns}:  sup = 0.184
{butter}:  sup = 0.055
{other vegetables}:  sup = 0.193
{pip fruit}:  sup = 0.076
{whole milk}:  sup = 0.256
{yogurt}:  sup = 0.14
{tropical fruit}:  sup = 0.105
{coffee}:  sup = 0.058
{margarine}:  sup = 0.059
{citrus fruit}:  sup = 0.083
{yogurt, other vegetables}:  sup = 0.043
{whole milk, rolls/buns}:  sup = 0.057
{whole milk, soda}:  sup = 0.04
{whole milk, root veg

In [29]:
# Generate the association rules from a list of frequent itemsets.
H = generate_rules(F, support_data, min_confidence=0.04, verbose=True)
# With 4% min_confidence factor we have got 18 rules which shows high co-occurence of items. 
# All rules are showing highly co-ocurrence 2 items.

# {root vegetables} ---> {whole milk}:  conf = 0.449, sup = 0.049
# {whole milk} ---> {root vegetables}:  conf = 0.191, sup = 0.049
#  The confidence difference between these 2 rules is huge. It is clear that association between these two items 
#  is one way i.e root vegetables -> whole milk

{other vegetables} ---> {yogurt}:  conf = 0.224, sup = 0.043
{yogurt} ---> {other vegetables}:  conf = 0.311, sup = 0.043
{rolls/buns} ---> {whole milk}:  conf = 0.308, sup = 0.057
{whole milk} ---> {rolls/buns}:  conf = 0.222, sup = 0.057
{soda} ---> {whole milk}:  conf = 0.23, sup = 0.04
{whole milk} ---> {soda}:  conf = 0.157, sup = 0.04
{root vegetables} ---> {whole milk}:  conf = 0.449, sup = 0.049
{whole milk} ---> {root vegetables}:  conf = 0.191, sup = 0.049
{root vegetables} ---> {other vegetables}:  conf = 0.435, sup = 0.047
{other vegetables} ---> {root vegetables}:  conf = 0.245, sup = 0.047
{tropical fruit} ---> {whole milk}:  conf = 0.403, sup = 0.042
{whole milk} ---> {tropical fruit}:  conf = 0.166, sup = 0.042
{other vegetables} ---> {rolls/buns}:  conf = 0.22, sup = 0.043
{rolls/buns} ---> {other vegetables}:  conf = 0.232, sup = 0.043
{whole milk} ---> {yogurt}:  conf = 0.219, sup = 0.056
{yogurt} ---> {whole milk}:  conf = 0.402, sup = 0.056
{other vegetables} ---> 

In [39]:
# Generate the association rules from a list of frequent itemsets.
H = generate_rules(F, support_data, min_confidence=0.40, verbose=True)
# Here on changing min confidence to 40%, we have found 4 rules.
# {yogurt} ---> {whole milk} rule has less confidence than rule {root vegetables} ---> {whole milk} which is odd for me
# as people who eats yogurt likes whole milk as well still confidence of this rule is less than {yogurt} ---> {whole milk}
#  Though rule {root vegetables} ---> {whole milk} makes sense as people who are nutrient conscious tend to eat root 
#  vegetables and also likes to drink whole milk. 

{root vegetables} ---> {whole milk}:  conf = 0.449, sup = 0.049
{root vegetables} ---> {other vegetables}:  conf = 0.435, sup = 0.047
{tropical fruit} ---> {whole milk}:  conf = 0.403, sup = 0.042
{yogurt} ---> {whole milk}:  conf = 0.402, sup = 0.056
